In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY", "TOOLHOUSE_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True)

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

In [24]:
## Example: LLM with Toolhouse
## Include the following tools in the Toolhouse bundle: thp-system-design-assistant
## Sendgrid, Web Scraper, Code Interpreter, and Memory tools are included in this bundle.

import os
from openai import OpenAI
from toolhouse import Toolhouse
import traceback

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TOOLHOUSE_API_KEY = os.getenv("TOOLHOUSE_API_KEY")
MAX_ITERS = 10
th = Toolhouse(api_key=TOOLHOUSE_API_KEY, provider="openai")
openai = OpenAI(api_key=OPENAI_API_KEY)


def llm(
    prompt: str,
    model="gpt-4o-mini",
    system="You are a helpful AI assistant",
    intermediate_output=True,
    verbose=True,
) -> str:
    """Wrapper over Open AI LLM calls with Toolhouse Tools."""
    try:
        if verbose:
            print("System: ", system)
            print("Prompt: ", prompt)
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ]
        response = openai.chat.completions.create(
            model=model,
            messages=messages,
            tools=th.get_tools("thp-system-design-assistant"),
        )
        messages += th.run_tools(response)

        tool_calls = response.choices[0].message.tool_calls
        content = response.choices[0].message.content
        iters = 0

        while tool_calls and len(tool_calls) > 0 and iters < MAX_ITERS:
            print("Tools: ", tool_calls)
            if verbose or intermediate_output:
                if content:
                    print(f"Cycle {iters}: ", content)
                else:
                    print(f"Cycle {iters}: No content.")
            response = openai.chat.completions.create(
                model=model,
                messages=messages,
                tools=th.get_tools("thp-system-design-assistant"),
            )
            messages += th.run_tools(response)
            if verbose or intermediate_output:
                print(f"Generated intermediate response...")
            content = response.choices[0].message.content
            tool_calls = response.choices[0].message.tool_calls
            iters += 1

        if content and (verbose or intermediate_output):
            print(f"Final response: {content}")

        return content
    except Exception as e:
        traceback.print_exc()
        return "Sorry, failed to generate response.\nError: " + str(e)

In [25]:
## Tesla Smart Routing Reporter

ROUTING_DATA = {
    "locations": [
        {"id": "A", "coordinates": [37.7749, -122.4194], "name": "San Francisco"},
        {"id": "B", "coordinates": [34.0522, -118.2437], "name": "Los Angeles"},
        {"id": "C", "coordinates": [36.7783, -119.4179], "name": "Fresno"},
    ],
    "traffic_data": [
        {
            "from": "A",
            "to": "C",
            "distance_km": 300,
            "traffic_level": "moderate",
            "speed_kmph": 80,
        },
        {
            "from": "C",
            "to": "B",
            "distance_km": 200,
            "traffic_level": "heavy",
            "speed_kmph": 50,
        },
        {
            "from": "A",
            "to": "B",
            "distance_km": 500,
            "traffic_level": "light",
            "speed_kmph": 100,
        },
    ],
    "elevation_data": [
        {"id": "A", "elevation_m": 16},
        {"id": "B", "elevation_m": 71},
        {"id": "C", "elevation_m": 94},
    ],
    "vehicle_specs": {
        "model": "Tesla Model 3",
        "efficiency_wh_per_km": 150,
        "battery_capacity_wh": 75000,
    },
}


prompts = [
    "Load this JSON dataset and display the locations, traffic, and elevation data in a readable format.",
    "Create a map visualizing the routes between the locations A, B, and C with distances and traffic levels as labels.",
    "Calculate the travel time for each route considering traffic and average speeds provided in the dataset. Provide the result in a readable format.",
    "Write a formula to calculate additional energy usage for elevation gains, assuming that for every 100 meters of elevation gain, the vehicle uses an additional 10 Wh per km.",
    "Compute the most energy-efficient route from San Francisco (A) to Los Angeles (B) using distance, traffic, speed, and elevation data.",
    "Provide a table comparing all possible routes between A and B with metrics like travel time, elevation gain, energy consumption (Wh), and remaining battery percentage.",
    "Simulate how changes in traffic levels or elevation impacts the energy consumption for the A to B route.",
    "Plot a bar chart showing energy consumption for each route between A and B.",
    "Create a heatmap showing locations reachable from A with 80% battery given the vehicle's specs.",
]

In [26]:
from IPython.display import Markdown, display
import json
import traceback

output_file = "test_output/tesla_routing_output.md"  # change as required


def is_section_in_file(section_hint: str, file_path: str) -> bool:
    """Check if a section hint is present in a file."""
    try:
        with open(file_path, "r") as f:
            content = f.read()
        return section_hint in content
    except Exception as e:
        print("Error reading file:", str(e))
        traceback.print_exc()
        return False


def create_file_if_not_exists_recursive(file_path: str):
    """Create a file if it does not exist, creating parent directories if necessary."""
    try:
        if not os.path.exists(file_path):
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            with open(file_path, "w") as f:
                f.write("")
    except Exception as e:
        print("Error creating file:", str(e))
        traceback.print_exc()

In [ ]:
create_file_if_not_exists_recursive(output_file)

for prompt in prompts:
    if is_section_in_file(prompt, output_file):
        print(f"Skipping prompt: {prompt}")
        continue
    data_json = json.dumps(ROUTING_DATA)
    prompt = f"{prompt}. Provide the response in a readable format.\n\n```json\n{data_json}\n```"
    print(f"Prompt: {prompt}")
    response = llm(prompt=prompt)
    print("Response: ", response)
    if not is_section_in_file(prompt, output_file):
        with open(output_file, "a") as f:
            f.write(f"## {prompt}\n" + response + "\n\n")
    print("\n\n")

print(
    f"🚀 All prompts completed. Check the output file ('{output_file}') for the responses."
)